In [1]:
import numpy as np
import xarray as xr
import pickle
import xesmf as xe

In [2]:
# Load quarterly degree data of gph and temperature
#infile = open("./500GPH/FULL500_ANOM_capgph1959.p", 'rb') 
infile = open("./data/cap_pv100hPa.p", 'rb') 
pvu = pickle.load(infile)
infile.close()

In [3]:
##indices
infile = open("era5_latindex.p", 'rb') 
lats = pickle.load(infile)
infile.close()

infile = open("era5_lonindex.p", 'rb') 
lons = pickle.load(infile)
infile.close()

In [4]:
time = [i for i in range(0,182)]

In [5]:
pvu.shape

(63, 152, 16, 180)

In [22]:
##save year index
year = [i for i in range(1959,2022)] #indicate years for data, needed for opening files

In [23]:
##save lat lon indices
pvu_lats = np.arange(90,58,-2)
pvu_lons = np.arange(0,360,2)
#temp_lats = lats[60:121]
#temp_lons = lons [760:901]

In [24]:
len(pvu_lons)

180

In [25]:
##create array for gph ... 
pv = xr.DataArray(data= pvu, dims = ["year","day","lat","lon"],coords = dict(year = year,
                                                                                day = time,
                                                                                lat = pvu_lats,
                                                                                lon = pvu_lons))
pv

<xarray.DataArray (year: 63, day: 182, lat: 16, lon: 180)>
array([[[[38.689137  , 38.689137  , 38.689137  , ..., 38.689137  ,
          38.689137  , 38.689137  ],
         [40.79346036, 40.88472997, 40.93874668, ..., 40.55271347,
          40.6365325 , 40.72640513],
         [40.68123599, 40.62116568, 40.60905849, ..., 41.14689727,
          41.06773486, 40.91453229],
         ...,
         [30.70304592, 30.42085518, 30.21922385, ..., 30.34495239,
          31.51981582, 30.89722668],
         [32.45626067, 32.06184556, 31.63576548, ..., 31.96405669,
          31.65858288, 31.3950186 ],
         [31.95334648, 32.68862565, 32.63367762, ..., 30.86556171,
          30.51957538, 31.37173553]],

        [[39.64310963, 39.64310963, 39.64310963, ..., 39.64310963,
          39.64310963, 39.64310963],
         [38.63402162, 38.63448728, 38.60002835, ..., 38.56277544,
          38.58326454, 38.60561628],
         [39.49828897, 39.67244629, 39.91645281, ..., 39.49223537,
          39.45498247, 39.5592906 ],
...
         [42.58286572, 48.15496868, 54.41252506, ..., 30.37788338,
          34.57069761, 36.95208943],
         [39.79169196, 45.6720627 , 51.12775034, ..., 30.99069363,
          33.26312071, 35.12297189],
         [35.28502202, 42.23734504, 48.67837197, ..., 29.44004154,
          32.37836427, 33.01445759]],

        [[33.93295265, 33.93295265, 33.93295265, ..., 33.93295265,
          33.93295265, 33.93295265],
         [34.32597077, 34.88290167, 35.56649244, ..., 33.93015868,
          33.98417539, 34.09500277],
         [35.82819409, 35.92505163, 36.04984886, ..., 34.66683484,
          35.04774577, 35.47242886],
         ...,
         [55.39341873, 57.79809362, 56.54639608, ..., 29.41045022,
          34.86054993, 46.57193131],
         [55.39900667, 57.19366527, 56.83510608, ..., 29.36295277,
          35.30851609, 47.26856059],
         [53.57361442, 56.66933066, 56.2046007 , ..., 28.34408588,
          33.99162597, 45.86319483]]]])
Coordinates:
  * year     (year) int64 1959 1960 1961 1962 1963 ... 2017 2018 2019 2020 2021
  * day      (day) int64 0 1 2 3 4 5 6 7 8 ... 174 175 176 177 178 179 180 181
  * lat      (lat) int64 90 88 86 84 82 80 78 76 74 72 70 68 66 64 62 60
  * lon      (lon) int64 0 2 4 6 8 10 12 14 ... 344 346 348 350 352 354 356 358

In [26]:
pv_out = xr.Dataset({'year':(['year'], year[:]),
                    'day':(['day'], time[:]),
                    'lat': (['lat'], np.arange(90, 62.5, -2.5)),
                    'lon': (['lon'], np.arange(0, 360.5, 2.5))})

In [27]:
# Create the regridder
regridder = xe.Regridder(pv, pv_out, 'bilinear', periodic=True)

In [28]:
# Regrid the data
pv_degree = regridder(pv)
pv_degree

<xarray.DataArray (year: 63, day: 182, lat: 11, lon: 145)>
array([[[[38.689137  , 38.689137  , 38.689137  , ..., 38.689137  ,
          38.689137  , 38.689137  ],
         [40.76540213, 40.82816576, 40.86730391, ..., 40.72290351,
          40.76620065, 40.76540213],
         [40.01999696, 40.00766018, 40.07735843, ..., 40.27326729,
          40.13059242, 40.01999696],
         ...,
         [37.95013254, 37.64206513, 37.24102511, ..., 37.89604274,
          37.92105011, 37.95013254],
         [32.65333643, 33.28325559, 33.56466786, ..., 32.64532966,
          32.24815805, 32.65333643],
         [31.37522799, 31.78837541, 32.23835758, ..., 31.20469761,
          31.0711146 , 31.37522799]],

        [[39.64310963, 39.64310963, 39.64310963, ..., 39.64310963,
          39.64310963, 39.64310963],
         [38.85010491, 38.90294611, 38.93420481, ..., 38.79835563,
          38.8334747 , 38.85010491],
         [39.38024383, 39.52774489, 39.73580918, ..., 39.24805809,
          39.31114019, 39.38024383],
...
         [41.25293708, 50.84197995, 55.38474648, ..., 32.05971102,
          35.299175  , 41.25293708],
         [43.10581778, 50.6808956 , 57.28385954, ..., 32.66044213,
          36.68471354, 43.10581778],
         [43.12862075, 50.21055744, 58.98240666, ..., 32.62350019,
          36.60681393, 43.12862075]],

        [[33.93295265, 33.93295265, 33.93295265, ..., 33.93295265,
          33.93295265, 33.93295265],
         [34.7015552 , 35.2795685 , 35.95222365, ..., 34.18238616,
          34.39224286, 34.7015552 ],
         [35.62982238, 35.93645613, 36.44311044, ..., 34.43065438,
          35.08191732, 35.62982238],
         ...,
         [46.81221253, 54.42098003, 55.80905568, ..., 30.97431271,
          36.86879433, 46.81221253],
         [50.25653967, 55.30529826, 55.69229318, ..., 31.53417464,
          39.89140193, 50.25653967],
         [54.07420031, 57.30028283, 56.71416776, ..., 31.9930199 ,
          42.69840466, 54.07420031]]]])
Coordinates:
  * year     (year) int64 1959 1960 1961 1962 1963 ... 2017 2018 2019 2020 2021
  * day      (day) int64 0 1 2 3 4 5 6 7 8 ... 174 175 176 177 178 179 180 181
  * lon      (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 352.5 355.0 357.5 360.0
  * lat      (lat) float64 90.0 87.5 85.0 82.5 80.0 ... 75.0 72.5 70.0 67.5 65.0
Attributes:
    regrid_method:  bilinear

In [29]:
pvu1 = pv_degree.values
pvu1.shape

(63, 182, 11, 145)

In [30]:
##remove leap days now
pvu1 = np.delete(pvu1,[150],1)

In [31]:
pickle.dump(pvu1, open("pvu100_25deg.p", 'wb'))

In [32]:
pvu1.shape

(63, 181, 11, 145)